# New Path Finder Code Snippets
Jeremy Johnson

The code below runs in Google Earth Engine, but not in this notebook. Most of this code is stored in my own modules which then are used in concert to train the model

In [ ]:
## Imports Google Earth Engine 
import ee

In [ ]:
#define the scene
LandSatScene = # defined var path like'LANDSAT/LC08/C01/T1/LC08_044034_20140318'

#import the scene
var raw = ee.Image(LandSatScene);
Map.centerObject(raw, 10); #for display
Map.addLayer(raw, {bands: ['B4', 'B3', 'B2'], min: 6000, max: 12000}, 'raw'); #for hypothetical display

#Map.addLayer(raw, {bands: ['B2', 'B5', 'B6'], min: 6000, max: 12000}, 'raw'); this is the best false color for my purposes
# B2 =  0.452 - 0.512   Bathymetric mapping
# B5 =  0.851 - 0.879
# B6 = 1.566 - 1.651  penetrates light cloud cover and provides moister index

#Convert the raw data to radiance.  This step isn't always necessary, but for my purposes it makes the most sense.
var radiance = ee.Algorithms.Landsat.calibratedRadiance(raw);
Map.addLayer(radiance, {bands: ['B4', 'B3', 'B2'], max: 90}, 'radiance');

Clouds are the mortal enemy of Remote Sensing, so the first "treatment" we need to apply is cloud identification and orthorectification

In [ ]:
### Load a cloudy Landsat scene and display it.
var cloudy_scene = ee.Image(LandSatScene);
Map.centerObject(cloudy_scene);
Map.addLayer(cloudy_scene, {bands: ['B4', 'B3', 'B2'], max: 0.4}, 'TOA', false);

### Add a cloud score band.  It is automatically called 'cloud'.  We will use this score later to throw out images
var scored = ee.Algorithms.Landsat.simpleCloudScore(cloudy_scene);

## Create a mask from the cloud score and combine it with the image mask.
var mask = scored.select(['cloud']).lte(20);

### Apply the mask to the image and display the result.
var masked = cloudy_scene.updateMask(mask);
Map.addLayer(masked, {bands: ['B4', 'B3', 'B2'], max: 0.4}, 'masked');

### Load a Landsat 8 composite and set the SENSOR_ID property.
var mosaic = ee.Image(ee.ImageCollection(collection).first())
  .set('SENSOR_ID', 'OLI_TIRS');

### this is the displac e Cloud score the mosaic and display the result.
var scored_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic);
Map.addLayer(scored_mosaic, {bands: ['B4', 'B3', 'B2'], max: 0.4},
    'TOA mosaic', false);

So now we need to account for the clouds and noise by making composite images from collections if possible

In [2]:
var collection = ee.ImageCollection(LandSatScene)
    .filterDate('2010-01-01', '2010-3-31');  # for my model, we look at 2 and 3 month composites to determine landcover change. 

###Create a cloud-free composite with default parameters.
var composite = ee.Algorithms.Landsat.simpleComposite(collection);

###Create a cloud-free composite with custom parameters for
###cloud score threshold and percentile.
var customComposite = ee.Algorithms.Landsat.simpleComposite({
  collection: collection,
  percentile: 75,
  cloudScoreRange: 5
});

### Display the composites. This also helps a processing sanity check
Map.setCenter(Coordinates, 10);
Map.addLayer(composite, {bands: ['B4', 'B3', 'B2'], max: 128}, 'TOA composite');
Map.addLayer(customComposite, {bands: ['B4', 'B3', 'B2'], max: 128},
    'Custom TOA composite');

# Supervised Classification 

In [ ]:
-#Use these bands for prediction.
var bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'];

# Load a Landsat 8 image to be used for prediction.
var image = ee.Image(LandSatScene) # or composite here
    .select(bands);

#    Load training points. The numeric property 'class' stores known labels.
var points = ee.FeatureCollection(Training_Feature)
    .remap([1, 2], [0, 1], 'class');

#  This Overlays the points on the imagery to get training.
var training = image.sampleRegions({
  collection: points,
  properties: ['class'],
  scale: 30
});

# Train a CART classifier with default parameters.
var trained = ee.Classifier.cart().train(training, 'class', bands);

# Classify the image with the same bands used for training.
var classified = image.select(bands).classify(trained);

# Display the inputs and the results.
Map.centerObject(image, 10);
Map.addLayer(image, {bands: ['B4', 'B3', 'B2'], max: 0.4}, 'image');
Map.addLayer(classified, {min: 0, max: 1, palette: ['00FF00', 'FF0000']},
  'classification');